In [25]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import requests

from utils import get_projects_mapping, get_all_projects_as_dataframe, get_units_dataframe

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_projects_keep = pd.read_csv(
    "./input/Apr2025/RemainingProject.csv",
    header=None,
    delimiter=",",
    encoding="latin1"  # or encoding="ISO-8859-1"
)
df_projects_keep

,0,1,2,3,4,5,6
0,Registered,Project title,Host parties,Other Parties,Methodology,Reductions,Ref
1,18/11/2004,Brazil NovaGerar Landfill Gas to Energy Project,Brazil,Netherlands/ Italy/ Luxembourg/ Switzerland/ J...,AM0003,670133,0008
2,11/01/2005,RIO BLANCO Small Hydroelectric Project,Honduras,Finland,AMS-I.D. ver. 4,17800,0028
3,08/03/2005,Project for GHG emission reduction by thermal ...,India,Switzerland/ Japan/ Netherlands/ Italy/ United...,AM0001 ver. 2,3000000,0001
4,24/03/2005,HFC Decomposition Project in Ulsan,Republic of Korea,Switzerland/ Japan,AM0001 ver. 2,1400000,0003
...,...,...,...,...,...,...,...
8233,NaN,"Low head Hydropower Development Project, Punja...",Pakistan,NaN,ACM0002 ver. 20,67150,10000
8234,NaN,Guadalupe Solar PV Project,Chile,NaN,AMS-I.D. ver. 18,8674,10732
8235,NaN,El Popal 20 MW hydroelectric project,Colombia,NaN,ACM0002 ver. 20,54765,10742
8236,NaN,Energy efficient equipment in mix use LE PARC ...,Ecuador,NaN,AMS-II.C. ver. 15,1768,10749


In [3]:
df_activities= pd.read_excel('./input/Apr2025/CDM-Activities-April2025.xlsx')


In [4]:
df_activities

,CDM project reference number,Unique project identifier (traceable with Google),Registration project title,Type of CDM project: PA/PoA,Project classification,Sectoral scope number(s),Methodologies used at registration,Project type (UNEP DTU),Project subtype (UNEP DTU),DOE,...,Issuance (Para 218)/ Registration (Para 219),Last information provided on para. 218 and 219 of the PCP,Date of last information provided on Para. 218 and 219 of the PCP,Number of CPAs per host party (PoA),Amount of reduction per host party (PoA),Crediting type of the CPAs,Chronological number of registered CDM activity by country,Last communication with the Secretariat,Projects which cannot be renewed,Reporting date
0,1.0,AuxID,Project for GHG emission reduction by thermal ...,PA,LARGE,11,AM0001 2.0,HFCs,HFC23,SGS United Kingdom Limited (SGS),...,NaN,NaN,NaT,NaN,NaN,NaN,IN(1),2022m10,No,10Oct2024
1,3.0,LJ80OQZR2JVOLTZZX4Y0EGB1N18AVO,HFC Decomposition Project in Ulsan,PA,LARGE,11,AM0001 2.0,HFCs,HFC23,Japan Quality Assurance Organisation (JQA),...,NaN,NaN,NaT,NaN,NaN,NaN,KR(1),2016m11,No,10Oct2024
2,8.0,SQA16OAS75TKWYZTTKK7G1LO0O8CMU,Brazil NovaGerar Landfill Gas to Energy Project,PA,LARGE,13,AM0003 4.0,Landfill gas,Landfill power,Det Norske Veritas- CUK,...,NaN,NaN,NaT,NaN,NaN,NaN,BR(1),2022m7,No,10Oct2024
3,9.0,HG20LFVJPFURJJ4M6E6MS07HTOZTR4,La Esperanza Hydroelectric Project,PA,SMALL,1,AMS-I.D. 11.0,Hydro,Run of river,Det Norske Veritas- CUK,...,NaN,NaN,NaT,NaN,NaN,NaN,HN(4),2021m12,No,10Oct2024
4,11.0,V3ZBEVSXGFNT31E2CECR2NYE93FW5L,Project for GHG Emission Reduction by Thermal ...,PA,LARGE,11,AM0001 3.0,HFCs,HFC23,Japan Quality Assurance Organisation (JQA),...,NaN,NaN,NaT,NaN,NaN,NaN,CN(10),2020m11,Yes,10Oct2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13168,NaN,TDMRK8WORSJS8PS0BBCZVUI8KGDIBG,Tecate Hydropower project,PA,LARGE,1,ACM0002,Hydro,New dam,Bureau Veritas Certification Holding SAS (BVCH),...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,2011m11,NaN,10Oct2024
13169,NaN,0KE428B5HBMSAQLPQ58O6509M6NIME,Waste heat recovery by implementing Heat Re-cl...,PA,LARGE,1; 4,ACM0012,EE supply side,Higher efficiency using waste heat,Det Norske Veritas- CUK,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,2008m5,NaN,10Oct2024
13170,NaN,45EPBL70YRK3JN28KE82ZFNE3SKUX7,India - Vertical Shaft Brick Kiln Cluster Proj...,PA,SMALL,3; 4,AMS-II.D.,EE Industry,Building materials,SGS United Kingdom Limited (SGS),...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,2008m6,NaN,10Oct2024
13171,NaN,3I96I5K6CL7810JG5G1OEZIZKS7S04,Serra dos Antunes wind farm,PA,LARGE,1,ACM0002,Wind,Wind,Spanish Association for Standardisation and Ce...,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,2009m2,NaN,10Oct2024


In [5]:
len(df_projects_keep[6].unique())

8238

In [6]:
# Convert 'ref' to int, then zero-padded string
df_activities = df_activities[
    df_activities['CDM project reference number'].notna() &
    (df_activities['CDM project reference number'].astype(str).str.lower() != 'nan')
]
# ...existing code...
import numpy as np

# Replace non-numeric values with NaN, then convert to float
df_projects_keep[6] = pd.to_numeric(df_projects_keep[6], errors='coerce')
print(df_projects_keep[6].unique())

# ...existing code...

[       nan 8.0000e+00 2.8000e+01 ... 1.0742e+04 1.0749e+04 1.0743e+04]


In [7]:
df_activities['CDM project reference number'] = pd.to_numeric(
    df_activities['CDM project reference number'], errors='coerce'
)
# Find matches
matches = df_projects_keep[6].isin(df_activities['CDM project reference number'])
num_matches = matches.sum()
print(f"Number of matching rows: {num_matches}")

Number of matching rows: 8233


/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_2371/1010606616.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activities['CDM project reference number'] = pd.to_numeric(


In [8]:
# ...existing code...

# Filter df_activities to keep only matching rows
df_activities_matched = df_activities[
    df_activities['CDM project reference number'].isin(df_projects_keep[6])
]
print(df_activities_matched.shape)
df_activities_matched.columns

# ...existing code...


(8233, 65)


Index(['CDM project reference number',
       'Unique project identifier (traceable with Google)',
       'Registration project title', 'Type of CDM project: PA/PoA',
       'Project classification', 'Sectoral scope number(s)',
       'Methodologies used at registration', 'Project type (UNEP DTU)',
       'Project subtype (UNEP DTU)', 'DOE', 'Start of validation',
       'PA:Initial registration request/PoA:First CPA request',
       'PA:Lastest registration request/PoA:Lastest CPA request',
       'Date of EB decision (see EB59 Annex 12, §25)',
       'Registered or rejected', 'Simplified project status',
       'Website project status', 'Type of crediting period',
       'Start of first crediting period', 'End of first crediting period',
       'Amount Of Reductions (PDD ex-ante) per year in CP1 (PoA based on the sum of curr',
       'Status of the second CP request',
       'Second crediting period: renew date',
       'Second crediting period: renewed date',
       'Amount Of Reduc

In [13]:

get_projects_url_local = "https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985"


project_id_to_warehouseId, issuance_mapping, locations_mapping,duplicate_projects_api,project_id_to_locations= get_projects_mapping(get_projects_url_local)


print(len(project_id_to_warehouseId))
print(len(issuance_mapping))
print(len(locations_mapping))
print(len(duplicate_projects_api))


https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=1&limit=1000
all good
<Response [200]>
https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=2&limit=1000
all good
<Response [200]>
https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=3&limit=1000
all good
<Response [200]>
https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=4&limit=1000
all good
<Response [200]>
https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=5&limit=1000
all good
<Response [200]>
https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=6&limit=1000


In [14]:
project_id_to_warehouseId

{'FYPIZCQJOZJNRXJA7SX0PSYRLGO94G': 'fecbec50-8c8b-45a3-b017-5f5fb3835413',
 'IKMQQ2OSVS8ABEPEZRAFY9AERSEKH5': 'e449a8c0-778b-4b63-ada3-b1c410c1b522',
 'ZYSW3EJW8IF7GDI6MYFA5KYCA84DD8': 'f18ebb58-ba73-48f3-b792-4155563f34d5',
 '6EHY7CURQ9DDKZNZCVTW9QH3RS1AWL': 'c1d3f319-d352-4f93-a0bf-334f6dcecb8d',
 '49CTOPGOOGIMRZOF5X0VILKX1OTYC3': '90ca64ef-cca2-4cd6-8752-98b5862a8640',
 'AVI4PLU4LHD091N4XZ7W9YZM6RBRG0': '2f916112-6c62-4e73-bb65-4d6169190607',
 'SC016Y5SBGMTU1XS6QYRQ8XF7H0QRO': 'f6e93c0d-7967-43c3-ae06-a89ff2253ba0',
 '97VY4TYOVDSC3TDCQP09BW9LOLRL7V': '98fc2ee1-4349-444f-9749-e0df07c8fea3',
 'TI6KZWRWSNBF6AKYZVQEQ482U3AJR1': '3910bfac-7f0c-4817-a1c8-72ef48892534',
 'H7DBOV5MLDIB4KD9Z8YFASZ9QU4YQ3': '3d987b4d-1b96-4cb9-aa5c-927fa04dd3e5',
 'HUQOLJOM8VWZZPSHNC7650DOALCOMM': '2290c477-0530-427f-8bc9-94ad00f12be3',
 'LTETSDLLVXGU306ZECFIISO11STV8P': 'a54ca927-4167-42b2-bf7f-1f2ea1ed6c9d',
 'U0A0CSRYC2DEDTHR67PF18E4N5BKC8': 'd5994923-bd3a-4aa5-9b91-f7cbf5ef2e1c',
 'P9KM30JU9ISCZX7O76C1RY4

In [15]:

matched_project_ids = set(df_activities_matched['Unique project identifier (traceable with Google)'])

print(len(matched_project_ids))
# Filter the dictionary
tobeDeletedProjectId = {
    k: v for k, v in project_id_to_warehouseId.items() if k not in matched_project_ids
}

print(f"Number of to-be-deleted project IDs: {len(tobeDeletedProjectId)}")


8233
Number of to-be-deleted project IDs: 4926


In [16]:

get_projects_url_local = "https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985"

df_all_projects = get_all_projects_as_dataframe(
get_projects_url_local)


Fetching: https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=1&limit=1000
Fetching: https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=2&limit=1000
Fetching: https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=3&limit=1000
Fetching: https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=4&limit=1000
Fetching: https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=5&limit=1000
Fetching: https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985&page=6&limit=1000
Fetching: https://observer.climateactiondata.org/api/v1/projects?orgUi

In [17]:
df_all_projects_cdm = df_all_projects[df_all_projects['currentRegistry'] == 'CDM Registry']
print(df_all_projects_cdm.shape)
df_all_projects_cdm.head()

(13154, 33)


,warehouseProjectId,orgUid,currentRegistry,projectId,originProjectId,registryOfOrigin,program,projectName,projectLink,projectDeveloper,...,description,createdAt,updatedAt,projectLocations,labels,issuances,coBenefits,relatedProjects,projectRatings,estimations
0,fecbec50-8c8b-45a3-b017-5f5fb3835413,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,FYPIZCQJOZJNRXJA7SX0PSYRLGO94G,FYPIZCQJOZJNRXJA7SX0PSYRLGO94G,CDM Registry,PA,Reforestation of Grazing Lands Hacienda E...,https://cdm.unfccc.int/Projects/Validation/DB/...,n.a.,...,Reforestation of Grazing Lands Hacienda E...,2023-12-11T04:14:31.000Z,2023-12-11T04:14:31.000Z,[{'id': 'ce402adc-8055-431b-acfe-742bafe6196d'...,[],[{'id': '1faf62ac-5502-419f-b05e-929bc142a872'...,[],[],[],[{'id': 'c0c207b5-0f53-4df6-884d-c14c70517ef5'...
1,e449a8c0-778b-4b63-ada3-b1c410c1b522,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,IKMQQ2OSVS8ABEPEZRAFY9AERSEKH5,IKMQQ2OSVS8ABEPEZRAFY9AERSEKH5,CDM Registry,PA,Small scale Jhansi JFM A/R CDM Project on degr...,https://cdm.unfccc.int/Projects/Validation/DB/...,n.a.,...,Small scale Jhansi JFM A/R CDM Project on degr...,2023-12-11T04:14:35.000Z,2023-12-11T04:14:35.000Z,[{'id': '4517c4e5-9058-4fde-a3f0-91e295af0d18'...,[],[{'id': '2cb06b27-aece-4606-a149-b72eeeb21c24'...,[],[],[],[{'id': '0c836e26-ca52-406c-84b9-f953c84493e8'...
2,f18ebb58-ba73-48f3-b792-4155563f34d5,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,ZYSW3EJW8IF7GDI6MYFA5KYCA84DD8,ZYSW3EJW8IF7GDI6MYFA5KYCA84DD8,CDM Registry,PA,Small scale Obra JFM A/R CDM Project on degrad...,https://cdm.unfccc.int/Projects/Validation/DB/...,n.a.,...,Small scale Obra JFM A/R CDM Project on degrad...,2023-12-11T04:14:29.000Z,2023-12-11T04:14:29.000Z,[{'id': 'b82f16b3-4f5a-4f7b-a2d4-5a1c6b463488'...,[],[{'id': '7dbcc84e-789a-43d6-8e71-9a11f057f5d6'...,[],[],[],[{'id': '89752386-3d47-4d3c-afb5-43f72dc08d75'...
3,c1d3f319-d352-4f93-a0bf-334f6dcecb8d,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,6EHY7CURQ9DDKZNZCVTW9QH3RS1AWL,6EHY7CURQ9DDKZNZCVTW9QH3RS1AWL,CDM Registry,PA,Chakshi Mini Hydro Power Project,https://cdm.unfccc.int/Projects/Validation/DB/...,n.a.,...,Chakshi Mini Hydro Power Project,2023-12-11T04:14:34.000Z,2023-12-11T04:14:34.000Z,[{'id': 'c655abf5-3619-4e00-ab2e-40049b73e6bf'...,[],[{'id': '064eb00f-de07-47c8-9e43-199c6df51ffa'...,[],[],[],[{'id': '01a97bc7-f8bf-43f7-bda6-147dd4cc9ff5'...
4,90ca64ef-cca2-4cd6-8752-98b5862a8640,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,49CTOPGOOGIMRZOF5X0VILKX1OTYC3,49CTOPGOOGIMRZOF5X0VILKX1OTYC3,CDM Registry,PoA,Gigawatt Global Programme of Activities,https://cdm.unfccc.int/Projects/Validation/DB/...,n.a.,...,Gigawatt Global Programme of Activities,2023-12-11T04:14:31.000Z,2023-12-11T04:14:31.000Z,[{'id': 'e2f2ccb1-0f62-4910-bc75-8c9b23f01c71'...,[],[{'id': '20663092-dd83-4279-b973-e833d534de4c'...,[],[],[],[{'id': '94f4673e-0d00-42e4-9d40-b89a737ef53e'...


In [18]:
import json

with open("tobeDeletedProjectId.json", "r") as f:
    tobeDeletedProjectId_loaded = json.load(f)

print(f"Loaded {len(tobeDeletedProjectId_loaded)} project IDs")

Loaded 4926 project IDs


In [19]:
df_deleted_projects = df_all_projects_cdm[
    df_all_projects_cdm['warehouseProjectId'].isin(tobeDeletedProjectId_loaded.values())
]

# df_deleted_projects.to_excel(
#     './output/Apr25/CDM-ToBeDeletedProjects-April2025.xlsx',
#     index=False
# )



In [4]:
df_deleted_projects =pd.read_excel('./output/Apr25/CDM-ToBeDeletedProjects-April2025.xlsx')
df_deleted_projects

,warehouseProjectId,orgUid,currentRegistry,projectId,originProjectId,registryOfOrigin,program,projectName,projectLink,projectDeveloper,...,description,createdAt,updatedAt,projectLocations,labels,issuances,coBenefits,relatedProjects,projectRatings,estimations
0,90ca64ef-cca2-4cd6-8752-98b5862a8640,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,49CTOPGOOGIMRZOF5X0VILKX1OTYC3,49CTOPGOOGIMRZOF5X0VILKX1OTYC3,CDM Registry,PoA,Gigawatt Global Programme of Activities,https://cdm.unfccc.int/Projects/Validation/DB/...,n.a.,...,Gigawatt Global Programme of Activities,2023-12-11T04:14:31.000Z,2023-12-11T04:14:31.000Z,[{'id': 'e2f2ccb1-0f62-4910-bc75-8c9b23f01c71'...,[],[{'id': '20663092-dd83-4279-b973-e833d534de4c'...,[],[],[],[{'id': '94f4673e-0d00-42e4-9d40-b89a737ef53e'...
1,f6e93c0d-7967-43c3-ae06-a89ff2253ba0,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,SC016Y5SBGMTU1XS6QYRQ8XF7H0QRO,SC016Y5SBGMTU1XS6QYRQ8XF7H0QRO,CDM Registry,PoA,Accelerating Electrification through Grid Exte...,https://cdm.unfccc.int/Projects/Validation/DB/...,Rural Electrification Agency Uganda,...,Accelerating Electrification through Grid Exte...,2023-12-11T04:14:35.000Z,2023-12-11T04:14:35.000Z,[{'id': '953833dc-ea75-44bd-8339-d0d851c31178'...,[],[{'id': '2f837311-4142-4f1b-a1c2-ed4022108356'...,[],[],[],[{'id': 'acd5f6e5-60c7-4e53-a8fe-2d3224ddb6f3'...
2,98fc2ee1-4349-444f-9749-e0df07c8fea3,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,97VY4TYOVDSC3TDCQP09BW9LOLRL7V,97VY4TYOVDSC3TDCQP09BW9LOLRL7V,CDM Registry,PoA,Biomass Energy Conservation Programme,https://cdm.unfccc.int/Projects/Validation/DB/...,n.a.,...,Biomass Energy Conservation Programme,2023-12-11T04:14:36.000Z,2023-12-11T04:14:36.000Z,[{'id': '622902ef-ef08-4d49-8f48-bd4ee52e7cbb'...,[],[{'id': '072af9b6-0925-4855-a4f9-5543ecfe841f'...,[],[],[],[{'id': '99fcf874-793d-43a6-a3c9-bd409696a573'...
3,2290c477-0530-427f-8bc9-94ad00f12be3,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,HUQOLJOM8VWZZPSHNC7650DOALCOMM,HUQOLJOM8VWZZPSHNC7650DOALCOMM,CDM Registry,PA,Capacity enhancement for export of surplus pow...,https://cdm.unfccc.int/Projects/Validation/DB/...,Maxambiental,...,Capacity enhancement for export of surplus pow...,2023-12-11T15:48:57.000Z,2023-12-11T15:48:57.000Z,[{'id': '30bc9773-8bbd-43b8-a3ec-d1766f1c7c94'...,[],[],[],[],[],[{'id': '8af3fdf9-6eee-47fb-b7b4-c7751dd057a9'...
4,a54ca927-4167-42b2-bf7f-1f2ea1ed6c9d,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,LTETSDLLVXGU306ZECFIISO11STV8P,LTETSDLLVXGU306ZECFIISO11STV8P,CDM Registry,PA,Energy efficient power generation at Dahej in ...,https://cdm.unfccc.int/Projects/Validation/DB/...,Adani Power,...,Energy efficient power generation at Dahej in ...,2023-12-11T15:49:03.000Z,2023-12-11T15:49:03.000Z,[{'id': '20e0c76b-b5ea-4279-87d3-d1ac308f3b2c'...,[],[],[],[],[],[{'id': 'e77c946a-4188-439e-96b7-cf59a80a3fe4'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4921,cfe2a1ed-88ea-4efa-ad2c-670e31b15a50,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,XCH8BCVGVGQBLUE1OWYT4ID47EN3XA,XCH8BCVGVGQBLUE1OWYT4ID47EN3XA,CDM Registry,PoA,Methane capture and combustion from Animal Was...,https://cdm.unfccc.int/Projects/Validation/DB/...,Sadia,...,Methane capture and combustion from Animal Was...,2023-12-11T14:54:29.000Z,2023-12-11T14:54:29.000Z,[{'id': '369fc331-82b7-4f57-a0d3-0fb133bafdef'...,[],[],[],[],[],[{'id': '5a73d54d-2752-433a-8ae9-c9fdbb50c0b5'...
4922,66fa893c-410d-4388-aec8-1562a2bbb66f,b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2...,CDM Registry,7VKAGAZ64KCANHRWZWCZMSU4AX1GGE,7VKAGAZ64KCANHRWZWCZMSU4AX1GGE,CDM Registry,PoA,CUIDEMOS Mexico (Campana De Uso Intelegente De...,https://cdm.unfccc.int/Projects/Validation/DB/...,Cool nrg Carbon Investments,...,CUIDEMOS Mexico (Campana De Uso Intelegente De...,2023-12-11T01:17:14.000Z,2023-12-11T01:17:14.000Z,[{'id': '54ded754-37f9-4bad-9c45-7d66c215c047'...,[],[{'id': '

NameError: name 'df_deleted_projects' is not defined

In [6]:
import ast

def extract_issuance_ids(df):
    """
    .
    """
    result = []
    for _, row in df.iterrows():
        warehouse_id = row['warehouseProjectId']
        issuances = row['issuances']
        # Parse if string
        if isinstance(issuances, str):
            try:
                issuances = ast.literal_eval(issuances)
            except Exception:
                issuances = []
        if isinstance(issuances, list):
            for obj in issuances:
                if isinstance(obj, dict) and 'id' in obj:
                    result.append({'warehouseProjectId': warehouse_id, 'issuanceId': obj['id']})
    return result

print(extract_issuance_ids(df_deleted_projects))
# Also print the length of the array, and then print the unique warehouseProjectId
issuance_ids = extract_issuance_ids(df_deleted_projects)
print("Total issuance records:", len(issuance_ids))
unique_warehouse_ids = set(item['warehouseProjectId'] for item in issuance_ids)
print("Unique warehouseProjectId:", unique_warehouse_ids)
print("Count of unique warehouseProjectId:", len(unique_warehouse_ids))


[{'warehouseProjectId': '90ca64ef-cca2-4cd6-8752-98b5862a8640', 'issuanceId': '20663092-dd83-4279-b973-e833d534de4c'}, {'warehouseProjectId': '90ca64ef-cca2-4cd6-8752-98b5862a8640', 'issuanceId': '6b1e265a-f355-4a65-9310-3a413e3a1887'}, {'warehouseProjectId': '90ca64ef-cca2-4cd6-8752-98b5862a8640', 'issuanceId': '9c366963-5de9-4426-8fff-35273ca8d8b9'}, {'warehouseProjectId': '90ca64ef-cca2-4cd6-8752-98b5862a8640', 'issuanceId': 'ac4f11d5-ba23-41e6-a331-085a3f47a13e'}, {'warehouseProjectId': '90ca64ef-cca2-4cd6-8752-98b5862a8640', 'issuanceId': 'df2b0c31-b909-4aef-95d7-1eb0ad843f18'}, {'warehouseProjectId': 'f6e93c0d-7967-43c3-ae06-a89ff2253ba0', 'issuanceId': '2f837311-4142-4f1b-a1c2-ed4022108356'}, {'warehouseProjectId': 'f6e93c0d-7967-43c3-ae06-a89ff2253ba0', 'issuanceId': '59c275dd-bab6-4366-bf67-069975e76e10'}, {'warehouseProjectId': 'f6e93c0d-7967-43c3-ae06-a89ff2253ba0', 'issuanceId': 'b96e7d94-2774-46ba-bba3-9243db069d4d'}, {'warehouseProjectId': 'f6e93c0d-7967-43c3-ae06-a89ff22

In [27]:

get_projects_url_local = "https://observer.climateactiondata.org/api/v1/projects?orgUid=b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985"
org_uid = "b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985"
get_units_url= "https://observer.climateactiondata.org/api/v1/units"


def get_units_dataframe(base_url, org_uid, limit=500):
    """
    Fetches all retired units for a given organization UID from a paginated API endpoint and returns them as a pandas DataFrame.
    """
    page = 1
    all_units = []
    while True:
        url = f"{base_url}?page={page}&limit={limit}&orgUid={org_uid}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            break
        data = response.json()
        units = data.get("data", [])
        retired_units = [unit for unit in units if unit.get("unitStatus") == "Issued"]
        all_units.extend(retired_units)
        if page >= data.get("pageCount", 0):
            break
        page += 1
    df_held_units = pd.DataFrame(all_units)
    return df_held_units
df_units = get_units_dataframe(get_units_url,org_uid)
# import json

# with open("tobeDeletedProjectId.json", "w") as f:
#     json.dump(tobeDeletedProjectId, f, indent=2)

# print("Saved tobeDeletedProjectId.json")

In [38]:
def filter_units_by_issuance_ids(df_units, issuance_ids):
	"""
	Returns rows from df_units whose issuanceId is in the list of issuance_ids.
	"""
	# Build a set of issuanceId for fast lookup
	issuance_id_set = set(item['issuanceId'] for item in issuance_ids)
	filtered = df_units[df_units['issuanceId'].isin(issuance_id_set)]
	return filtered

# Example usage:
filtered_units = filter_units_by_issuance_ids(df_units, issuance_ids)
print(filtered_units.shape)
filtered_units
# Sum the unit Counts from filtered_units['unitCount]
total_unit_count = filtered_units['unitCount'].astype(int).sum()
print(f"Total unitCount: {total_unit_count}")

# also export the filtered unit to excel
filtered_units.to_excel('./output/Apr25/FilteredUnitsByIssuanceId.xlsx', index=False)
print("Exported filtered units to './output/Apr25/FilteredUnitsByIssuanceId.xlsx'")


(443, 27)
Total unitCount: 66021165
Exported filtered units to './output/Apr25/FilteredUnitsByIssuanceId.xlsx'
